In [1]:
# create_strict_blue_mask.py
import cv2
import numpy as np

def jpg_to_npy_strict_blue(image_path, save_path="safe_grid_test.npy"):
    """
    Converts any image to a binary navigation grid:
    - ANY shade of blue (light, dark, cyan, turquoise...) → 0 (flooded/unsafe)
    - All other colors → 1 (safe land)
    Saves as .npy file for your path planner.
    """
    img = cv2.imread(image_path)
    if img is None:
        raise FileNotFoundError(f"Image not found: {image_path}")

    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Extremely broad blue detection — catches ALL blues
    blue_masks = [
        cv2.inRange(hsv, np.array([100, 30, 30]),   np.array([130, 255, 255])),  # main blue
        cv2.inRange(hsv, np.array([80,  30, 20]),   np.array([150, 255, 255])),  # wide blue/cyan
        cv2.inRange(hsv, np.array([90,  50, 50]),   np.array([140, 255, 255])),  # strong blue
        cv2.inRange(hsv, np.array([0,   0,  0]),    np.array([180, 255, 80])),   # very dark blues (low V)
    ]

    # Combine all blue detections
    final_blue_mask = blue_masks[0]
    for mask in blue_masks[1:]:
        final_blue_mask = cv2.bitwise_or(final_blue_mask, mask)

    # Create grid: blue → 0, non-blue → 1
    grid = np.where(final_blue_mask > 0, 0, 1).astype(np.uint8)

    # Save
    np.save(save_path, grid)
    print(f"Saved: {save_path}")
    print(f"   Shape: {grid.shape}")
    print(f"   Safe cells (1): {np.sum(grid == 1)}")
    print(f"   Flooded cells (0): {np.sum(grid == 0)}")

    # Optional: show preview
    preview = cv2.resize(img, (800, 800))
    mask_vis = cv2.cvtColor(final_blue_mask, cv2.COLOR_GRAY2BGR)
    mask_vis[final_blue_mask == 255] = [0, 0, 255]  # red overlay on blue
    combined = cv2.addWeighted(preview, 0.7, cv2.resize(mask_vis, (800,800)), 0.3, 0)
    cv2.imshow("Blue Detection Preview (Red = Detected Flood)", combined)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    return grid

# === RUN THIS TO CREATE YOUR MASK ===
if __name__ == "__main__":
    jpg_to_npy_strict_blue("flood_4.jpg", "safe_grid_4.npy")

Saved: safe_grid_4.npy
   Shape: (780, 1171)
   Safe cells (1): 599038
   Flooded cells (0): 314342
